# Chapter 6: Moments

## Functions

The MGF of an r.v. is a _function_. As an example of defining and working with functions in R, let's use the $\mathcal N(0,1)$ MGF, which is given by $M(t) = e^{t^2/2}$. The code

In [ ]:
M <- function(t) {exp(t^2/2)}

defines $M$ to be this function. The `function(t)` says that we're defining a function of one variable $t$ (called the _argument_ of the function). Then, for example, `M(0)` evaluates the function at $0$, `M(1:10)` evaluates the function at $1,2,\ldots,10$, and `curve(M,from=-3,to=3)` plots the graph of $M$ from $−3$ to $3$. Writing

In [ ]:
M <- function(x) {exp(x^2/2)}

would define the same function $M$, except that now the argument is named $x$. Giving the arguments names is helpful for functions of more than one variable, since R then saves us from having to remember the order in which to write the arguments, and allows us to assign default values. For example, the $\mathcal N(\mu, \sigma^2)$ MGF is given by $g(t) = \text{exp}(\mu t + \sigma^2 t^2 /2)$, which depends on $t$, $\mu$,and $\sigma$. We can define this in R by

In [ ]:
g <- function(t,mean=0,sd=1) {exp(mean*t + sd^2*t^2/2)}

What is `g(1,2,3)`? It's the $\mathcal N(2,3^2)$ MGF evaluated at $1$, but it may be hard to remember which argument is which, especially when working with many functions with many arguments over the course of many months. So we can also write `g(t=1,mean=2,sd=3)` or `g(mean=2,sd=3,t=1)` or any of the other $4$ permutations to mean the same thing.

Also, when defining $g$ we specified _default values_ of $0$ for the mean and $1$ for the standard deviation, so if we want the $\mathcal N(0,5^2)$ MGF evaluated at $3$, we can use `g(t=3,sd=5)` as shorthand. It would be bad here to write `g(3,5)`, since that is ambiguous about which argument is omitted; in fact, R interprets this as `g(t=3,mean=5)`.

## Moments

LOTUS makes it easy to _write down_ any moment of a continuous r.v. as an integral, and then R can help us _do_ the integral numerically, using the `integrate` command. For example, let's approximate the 6th moment of a $\mathcal N(0,1)$ r.v. The code

In [ ]:
g <- function(x) x^6*dnorm(x)
integrate(g, lower = -Inf, upper = Inf)

asks R to compute $\int_{-\infty}^\infty g(x) dx$, where $g(x) =x^6 \varphi(x)$ with $\varphi$ the $\mathcal N(0,1)$ PDF. When we ran this, R reported 15 (the correct answer, as we know from this chapter!) and that the absolute error was less than $7.9 \times 10^{−5}$. Similarly, to check that the 2nd moment (and variance) of a $\text{Unif}(−1,1)$ r.v. is $1/3$, we can use

In [ ]:
h <- function(x) x^2*dunif(x,-1,1)
integrate(h, lower = -1, upper = 1)

__WARNING__: Numerical integration runs into difficulties for some functions; as usual, checking answers in multiple ways is a good idea. Using `upper = Inf` is preferred to using a large number as the upper limit when integrating up to $\infty$ (and likewise for a lower limit of $-\infty$). For example, on many systems `integrate(dnorm,0,10^6)` reports $0$ while `integrate(dnorm,0,Inf)` reports the correct answer, 0.5.

For moments of a discrete r.v., we can use LOTUS and the `sum` command. For example, to find the 2nd moment of $X \sim \text{Pois}(7)$, we can use

In [ ]:
g <- function(k) k^2*dpois(k,7)
sum(g(0:100))

Here we summed up to $100$ since it's clear after getting a sense of the terms that the total contribution of all the terms after $k=100$ is negligible (choosing an upper limit in this way is in contrast to how to use the `integrate` command in the continuous case). The result is extremely close to $56$, which is comforting since $E(X^2) = \text{Var}(X) + (EX)^2 = 7 + 49 = 56$.

A sample moment can be found in one line in R. If `x` is a vector of data, then `mean(x)` gives its sample mean and, more generally, `mean(x^n)` gives the $n\text{th}$ sample mean for any positive integer $n$. For example,

In [ ]:
x <- rnorm(100)
mean(x^6)

gives the 6th sample moment of $100$ i.i.d. $\mathcal N(0,1)$ r.v.s. How close is it to the true 6th moment? How close are other sample moments to the corresponding true moments?

The sample variance can also be found in one line in R. If `x` is a vector of data, then `var(x)` gives its sample variance. This returns NA (not available) if `x` has length 1, since the $n−1$ in the denominator is $0$ in this case. It makes sense not to return a numerical value in this case, not only because of the definition but also because it would be insane to try to estimate the variability of a population if we only have one observation!

For a simple demonstration of using the sample mean and sample variance to estimate the true mean and true variance of a distribution, we generate $1000$ times from a $\mathcal N(0,1)$ distribution and store the values in `z`. We then compute the sample mean and sample variance with `mean` and `var`.

In [ ]:
z <- rnorm(1000)
mean(z)
var(z)

We find that `mean(z)` is close to $0$ and `var(z)` is close to $1$. You can try this out for a $\mathcal N(\mu, \sigma^2)$ distribution (or other distribution) of your choosing; just remember that `rnorm` takes $\sigma$ and not $\sigma^2$ as an input!

The sample standard deviation of `x` can be found using `sd(x)`. This gives the same result as `sqrt(var(x))`.

R does not come with built-in functions for sample skewness or sample kurtosis, but we can define our own functions as follows.

In [ ]:
skew <- function(x) {
    centralmoment <- mean((x-mean(x))^3)
    centralmoment/(sd(x)^3)
}

kurt <- function(x) {
    centralmoment <- mean((x-mean(x))^4)
    centralmoment/(sd(x)^4) - 3
}

## Medians and modes

To find the median of a continuous r.v. with CDF $F$, we need to solve the equation $F(x) = 1/2$ for $x$, which is equivalent to finding the root (zero) of the function $g$ given by $g(x) = F(x) - 1/2$. This can be done using `uniroot` in R. For example, let's find the median of the $\text{Expo}(1)$ distribution. The code

In [ ]:
g <- function(x) pexp(x) - 1/2
uniroot(g,lower=0,upper=1)

asks R to find a root of the desired function between $0$ and $1$. This returns an answer very close to the true answer of $\log(2) \approx 0.693$. Of course, in this case we can solve $1-e^{-x} = 1/2$ directly without having to use numerical methods.

__WARNING__: The `uniroot` command is useful but it only attempts to find _one_ root (as the name suggests), and there is no guarantee that it will find a root.

An easier way to find the median of the $\text{Expo}(1)$ in R is to use `qexp(1/2)`. The function `qexp` is the quantile function of the $\text{Expo}(1)$ distribution, which means that `qexp(p)` is the value of $x$ such that $P(X \leq x) = p$ for $X \sim \text{Expo}(1)$.

For finding the mode of a continuous distribution, we can use the `optimize` functionin R. For example, let's find the mode of the $\text{Gamma}(6,1)$ distribution, which is an important distribution that we will introduce in the next chapter. Its PDF is proportional to $x^5e^{-x}$. Using calculus, we can find that the mode is at $x= 5$. Using R, we can find that the mode is very close to $x= 5$ as follows.

In [ ]:
h <- function(x) x^5*exp(-x)
optimize(h,lower=0,upper=20,maximum=TRUE)

If we had wanted to minimize instead of maximize, we could have put `maximum=FALSE`.

Next, let's do a discrete example of median and mode. An interesting fact about the $\text{Bin}(n,p)$ distribution is that if the mean $np$ is an integer, then the median and mode are also $np$ (even if the distribution is very skewed). To check this fact about the median for the $\text{Bin}(50,0.2)$ distribution, we can use the following code.

In [ ]:
n <- 50; p <- 0.2
which.max(pbinom(0:n,n,p)>=0.5)

The `which.max` function finds the location of the maximum of a vector, giving the index of the _first_ occurrence of a maximum. Since TRUE is encoded as $1$ and FALSE is encoded as $0$, the first maximum in `pbinom(0:n,n,p)>=0.5` is at the first value for which the CDF is at least $0.5$. The output of the above code is $11$, but we must be careful to avoid an off-by-one error: the index $11$ corresponds to the median being 10, since we started evaluating the CDF at $0$. Similarly, `which.max(dbinom(0:n,n,p))` returns $11$, showing that the mode is at $10$.

The _sample median_ of a vector $\mathbf x$ of data can be found using `median(x)`. But `mode(x)` does not give the sample mode of $\mathbf x$ (rather, it gives information about what type of object $\mathbf x$ is). To find the sample mode (or sample modes, in case there are ties), we can use the following function.

In [ ]:
datamode <- function(x) {
    t <- table(x)
    m <- max(t)
    as.numeric(names(t[t==m]))
}

## Log-Normal and Weibull distributions

Analogous to the functions `dnorm`, `pnorm`, and `rnorm` for the Normal distribution, the functions `dlnorm`, `plnorm`, and `rlnorm` give the Log-Normal PDF, the Log-Normal CDF, and random generation of Log-Normal r.v.s, respectively. The parameters used for these functions for the Log-Normal are the mean and standard deviation of the underlying Normal.

For example, `dlnorm(x,1,2)` gives the PDF of the $\mathcal{LN}(1,4)$ distribution (not $\mathcal{LN}(1,2)$, nor a Log-Normal whose mean is 1). Because of the relationship between Normal and Log-Normal, `rlnorm(n,mu,sigma)` is equivalent to `exp(rnorm(n,mu,sigma))`.

For the Weibull, we can obtain the PDF, the CDF, and random generation with the functions `dweibull`, `pweibull`, and `rweibull`, respectively. The parametrization in R is different from the one we are using, but it is easy to convert between them: for the $\text{Wei}(\lambda, \gamma)$ distribution, let $a=\gamma$ and $b=\lambda^{-1/\gamma}$.

Then `dweibull(x,a,b)` gives the $\text{Wei}(\lambda,\gamma)$ PDF, `pweibull(x,a,b)` gives the CDF, and `rweibull(n,a,b)` generates $n$ i.i.d. draws from the distribution. Because of the relationship between Exponential and Weibull, another way to generate $\text{Wei}(\lambda,\gamma)$ r.v.s is to generate $\text{Expo}(\lambda)$ r.v.s and then raise each of them to the $1/\gamma$ power.

## Dice simulation

In the starred Section 6.7, we showed that in rolling $6$ fair dice, the probability of a total of $18$ is $3431/6^6 \approx 0.07354$. But the proof was complicated. If we only need an approximate answer, simulation is a much easier approach. And we already know how to do it! Here is the code for a million repetitions:

In [ ]:
r <- replicate(10^6,sum(sample(6,6,replace=TRUE)))
sum(r==18)/10^6

In our simulation this yielded $0.07346$, which is very close to $0.07354$.